## Using Virtual Tables

In [1]:
import sqlite3
import sqlite_vec

db_path = "/Users/abhinav/Desktop/shit/virtual_tables.db"
db = sqlite3.connect(db_path)
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)


In [2]:
# Create table without any embeddings column
# Create virtual tables for storing embeddings with vec0
db.execute(
    """
    CREATE TABLE goods (
        row_id INT PRIMARY KEY,
        Schedule VARCHAR(10),
        Serial_No INT,
        Chapter_Heading_Subheading_Tariff_Item TEXT,
        Description_of_Goods TEXT,
        CGST_Rate DECIMAL(5, 2),
        SGST_UTGST_Rate DECIMAL(5, 2),
        IGST_Rate DECIMAL(5, 2),
        Compensation_Cess VARCHAR(10)
    );
    """
)

In [3]:
items = [
    (1, 'I', 1, '0202, 0203, 0204, 0205, 0206, 0207, 0208, 0209, 0210', 'All goods [other than fresh or chilled] pre-packaged and labelled.', 2.5, 2.5, 5.0, None),
    (2, 'I', 2, '0303, 0304, 0305, 0306, 0307, 0308, 0309', 'All goods [other than fresh or chilled] pre-packaged and labelled.', 2.5, 2.5, 5.0, None),
    (3, 'I', 3, None, '[Omitted]', None, None, None, None),
    (4, 'I', 4, None, '[Omitted]', None, None, None, None),
    (5, 'I', 5, None, '[Omitted', None, None, None, None),
    (6, 'I', 6, None, '[Omitted]', None, None, None, None),
    (7, 'I', 7, '0401', 'Ultra High Temperature (UHT) milk', 2.5, 2.5, 5.0, None),
    (8, 'I', 8, '0402', 'Milk and cream, concentrated or containing added sugar or other sweetening matter, including skimmed milk powder, milk food for babies [other than condensed milk]', 2.5, 2.5, 5.0, None),
    (9, 'I', 9, '0403', 'Yoghurt; Cream, kephir and other fermented or acidified milk and cream, whether or not concentrated or containing added sugar or other sweetening matter or flavored or containing added fruit, nuts or cocoa', 2.5, 2.5, 5.0, None)
]

from models import Row
items = [Row(tuple=item) for item in items]

In [4]:
from indexer import SQLiteVecIndexer

indexer = SQLiteVecIndexer(db_path, using_virtual_table=True)
indexer.insert_rows(items)

In [ ]:
from retriever import SQLiteVecRetriever

query = "Give me about milk, cream, powder for babies"
retriever = SQLiteVecRetriever(db_path=db_path, using_virtual_table=True)
results = retriever.retrieve(query)
for i in results:
    print(i)

(8, 'I', 8, '0402', 'Milk and cream, concentrated or containing added sugar or other sweetening matter, including skimmed milk powder, milk food for babies [other than condensed milk]', 2.5, 2.5, 5, None)
(9, 'I', 9, '0403', 'Yoghurt; Cream, kephir and other fermented or acidified milk and cream, whether or not concentrated or containing added sugar or other sweetening matter or flavored or containing added fruit, nuts or cocoa', 2.5, 2.5, 5, None)
(7, 'I', 7, '0401', 'Ultra High Temperature (UHT) milk', 2.5, 2.5, 5, None)


## Using Normal Tables (Manually)

In [6]:
import sqlite3
import sqlite_vec

db_path = "/Users/abhinav/Desktop/shit/normal_tables.db"
db = sqlite3.connect(db_path)
db.enable_load_extension(True)
sqlite_vec.load(db)
db.enable_load_extension(False)

In [7]:
# Create table with embeddings column of type BLOB to store embeddings
db.execute(
    """
    CREATE TABLE goods (
        row_id INT PRIMARY KEY,
        Schedule VARCHAR(10),
        Serial_No INT,
        Chapter_Heading_Subheading_Tariff_Item TEXT,
        Description_of_Goods TEXT,
        CGST_Rate DECIMAL(5, 2),
        SGST_UTGST_Rate DECIMAL(5, 2),
        IGST_Rate DECIMAL(5, 2),
        Compensation_Cess VARCHAR(10),
        embeddings float[1536]
        check(
            typeof(embeddings) == 'blob'
            and vec_length(embeddings) == 1536
        )
    );
    """
)

In [8]:
items = [
    (1, 'I', 1, '0202, 0203, 0204, 0205, 0206, 0207, 0208, 0209, 0210', 'All goods [other than fresh or chilled] pre-packaged and labelled.', 2.5, 2.5, 5.0, None),
    (2, 'I', 2, '0303, 0304, 0305, 0306, 0307, 0308, 0309', 'All goods [other than fresh or chilled] pre-packaged and labelled.', 2.5, 2.5, 5.0, None),
    (3, 'I', 3, None, '[Omitted]', None, None, None, None),
    (4, 'I', 4, None, '[Omitted]', None, None, None, None),
    (5, 'I', 5, None, '[Omitted', None, None, None, None),
    (6, 'I', 6, None, '[Omitted]', None, None, None, None),
    (7, 'I', 7, '0401', 'Ultra High Temperature (UHT) milk', 2.5, 2.5, 5.0, None),
    (8, 'I', 8, '0402', 'Milk and cream, concentrated or containing added sugar or other sweetening matter, including skimmed milk powder, milk food for babies [other than condensed milk]', 2.5, 2.5, 5.0, None),
    (9, 'I', 9, '0403', 'Yoghurt; Cream, kephir and other fermented or acidified milk and cream, whether or not concentrated or containing added sugar or other sweetening matter or flavored or containing added fruit, nuts or cocoa', 2.5, 2.5, 5.0, None)
]

from models import Row
items = [Row(tuple=item) for item in items]

In [9]:
from indexer import SQLiteVecIndexer

indexer = SQLiteVecIndexer(db_path, using_virtual_table=False)
indexer.insert_rows(items)

In [ ]:
from retriever import SQLiteVecRetriever

query = "Give me about milk, cream, powder for babies"
retriever = SQLiteVecRetriever(db_path, using_virtual_table=False)
results = retriever.retrieve(query)
for i in results:
    print(i)

(8, 'I', 8, '0402', 'Milk and cream, concentrated or containing added sugar or other sweetening matter, including skimmed milk powder, milk food for babies [other than condensed milk]', 2.5, 2.5, 5, None)
(9, 'I', 9, '0403', 'Yoghurt; Cream, kephir and other fermented or acidified milk and cream, whether or not concentrated or containing added sugar or other sweetening matter or flavored or containing added fruit, nuts or cocoa', 2.5, 2.5, 5, None)
(7, 'I', 7, '0401', 'Ultra High Temperature (UHT) milk', 2.5, 2.5, 5, None)
